#### URL 이미지 불러오기

In [16]:
import requests
from PIL import Image, ImageDraw
from io import BytesIO
import gradio as gr


IMAGE_URL = "https://cdn.pixabay.com/photo/2023/12/13/22/29/young-woman-8447841_1280.jpg"
ENDPOINT_URL = "https://fimtrus-vision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze"
API_KEY = "1Xvo96LQT7LFAe6FgQUeO3f8UwOfELLcvZNJThBQW6LJsBdUhwzrJQQJ99BGACYeBjFXJ3w3AAAFACOGxEEM"


def random_color():
    import random
    # 랜덤한 RGB 색상 튜플 반환
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 

def get_font():
    # OS별로 적절한 폰트 객체 반환 (한글 지원)
    from PIL import ImageFont
    import platform
    
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            # 윈도우용 한글 폰트
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            # 맥용 한글 폰트
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux      
            # 리눅스 기본 폰트
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 폰트 파일이 없을 경우 기본 폰트 사용
        return ImageFont.load_default(size=font_size)
    

def request_image_analysis(image_url, features=["objects"], **kwargs):
    endpoint = "{}".format(ENDPOINT_URL)
    
    params = {
        "api-version": "2024-02-01",
        "features": ",".join(features)
    }
    
    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY
    }
    
    body = {
        "url": image_url
    }
    
    if kwargs is not None:
    
        if ("caption" in features or "denseCaptions" in features) and "gender_neutral_caption" in kwargs.keys() :
            params.update({
                "gender-neutral-caption": kwargs['gender_neutral_caption']
            })
            
        if "smartCrops" in features and "smartcrops_aspect_ratios" in kwargs.keys():
            params.update({
                "smartcrops-aspect-ratios": kwargs["smartcrops_aspect_ratios"]
            })
        
    response = requests.post(endpoint, params=params, headers=headers, json=body)
    print(response)
    if response.status_code != 200:
        return None
    
    response_json = response.json()
    
    return response_json
    
    

def draw_image(image_url, features, data):
    # 이미지 다운로드
    image_response = requests.get(image_url)
    image = Image.open(BytesIO(image_response.content))
    draw = ImageDraw.Draw(image)
    font = get_font()       # 폰트 객체

    for feature in features:
        
        if feature == "tags" or feature == "caption":
            continue
        
        feature_key = '{}Result'.format(feature)
        print(feature_key)
        # 객체 인식 결과 리스트
        result_object = data[feature_key]
        block_list = result_object['values']
        
        color = random_color()  # 랜덤 색상
        
        for block in block_list:
            
            bounding_box = block['boundingBox']
            x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']
            
            if feature == "objects":
                tag = block['tags'][0]
                name = tag['name']
                confidence = tag['confidence']
        
                formatted_text = "{} ({:.2f}%)".format(name, confidence * 100)
                
            elif feature == "denseCaptions":
                formatted_text = block["text"]
            else:
                formatted_text = None
            
            # 객체 위치에 사각형과 라벨 그리기
            draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=2)
            
            # Feature의 텍스트를 그려주는 부분
            feature_text_bbox = draw.textbbox((x, y), feature, font=font)            
            draw.rectangle([(feature_text_bbox[0], feature_text_bbox[1]),(feature_text_bbox[2], feature_text_bbox[3])], outline=color, fill=color, width=2)
            draw.text((x, y), feature, fill="black", font=font)
            
            # 텍스트를 그려주는 부분
            if formatted_text:
                text_bbox = draw.textbbox((x, y + 20), formatted_text, font=font)      
                draw.rectangle([(text_bbox[0], text_bbox[1]),(text_bbox[2] + 5, text_bbox[3])], outline=color, fill=color, width=2)
                draw.text((x + 5, y + 20), formatted_text, fill="white", font=font)
    
    return image

theme = gr.themes.Soft(primary_hue=gr.themes.colors.indigo)
with gr.Blocks(theme=theme) as demo:
    
    def click_send(image_url):
        features = ["objects", "caption", "denseCaptions", "tags", "smartCrops"]
        response_data = request_image_analysis(image_url, features)
        image = draw_image(image_url, features, response_data)
        
        return image
    
    image_url_textbox = gr.Textbox(label="이미지 URL")
    send_button = gr.Button("전송")
    output_image = gr.Image(label="결과 화면", interactive=False, type="pil")
    
    send_button.click(click_send, inputs=[image_url_textbox], outputs=[output_image])

demo.launch()

# 객체 인식 결과 요청 및 시각화
# features = ["objects", "caption", "denseCaptions", "tags", "smartCrops"]
# response_json2 = request_image_analysis(IMAGE_URL, features, gender_neutral_caption=False, smartcrops_aspect_ratios="0.75,1.8")
# draw_image(IMAGE_URL, features, response_json2)
# response_json2
# draw_image(IMAGE_URL, response_json2)




* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [39]:
features = ["objects", "caption", "denseCaptions", "tags", "smartCrops"]
for feature in features:
    print('{}Result'.format(feature))

objectsResult
captionResult
denseCaptionsResult
tagsResult
smartCropsResult


In [62]:
image_path = "/Users/fimtrus/Downloads/merry-christmas-4697055_1280.jpg"
image_path.startswith('http')

False